In [63]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os

import matplotlib.pyplot as plt

rng = 69

In [64]:
df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Transformed_PCA - Test v1.csv")

In [65]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away'])

In [66]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [67]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [68]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

win_or_lose_Home             1.000000
seasonPointsPerGame_Home     0.301171
fenwickPercentageAvg_Home    0.158833
goalDiffAvg_Home             0.111469
pointsFromGameAvg_Home       0.105522
hitsDiffAvg_Away             0.074194
reboundsForAvg_Home          0.059812
penaltiesAgainstTotal        0.028012
penaltiesForTotal           -0.019402
hitsDiffAvg_Home            -0.038123
reboundsForAvg_Away         -0.057037
pointsFromGameAvg_Away      -0.112578
goalDiffAvg_Away            -0.113146
fenwickPercentageAvg_Away   -0.164881
seasonPointsPerGame_Away    -0.292426
Name: win_or_lose_Home, dtype: float64

In [69]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [70]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [71]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_or_lose_Home', axis=1)
nhl_labels = train_set['win_or_lose_Home'].copy()

X_train = train_set.drop('win_or_lose_Home', axis=1)
Y_train = train_set['win_or_lose_Home'].copy()

X_valid = train_set.drop('win_or_lose_Home', axis=1)
Y_valid = train_set['win_or_lose_Home'].copy()

X_test = test_set.drop('win_or_lose_Home', axis=1)
Y_test = test_set['win_or_lose_Home'].copy()

df_Final = df_Final.drop('win_or_lose_Home', axis=1)

In [72]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [73]:
# Fine Tune the hyperparameters of RandomForest model
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},
#               {'bootstrap': [False], 'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},]

# rnd_clf_test = RandomForestClassifier()

# grid_search = GridSearchCV(rnd_clf_test, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# grid_search.fit(nhl_prepared,nhl_labels)

# grid_search.best_params_

In [74]:
# cvres = grid_search.cv_results_
# results = []
# for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
#     results.append([np.sqrt(-mean_scores),params])

# results2 = sorted(results, key=lambda x: x[0])
# print(*results2, sep = '\n')

In [75]:
# baseline model
from tensorflow import keras


# create model
base_model = keras.models.Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
                       ])
    
# Compile model
base_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [76]:
from scikeras.wrappers import KerasClassifier


keras_clf = KerasClassifier(model=base_model, epochs=20, batch_size=5, verbose=0)
keras_clf.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid), random_state=42)



KerasClassifier(
	model=<Sequential name=sequential_10, built=True>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=20
	class_weight=None
)

In [77]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

#kfold = StratifiedKFold(n_splits=5, shuffle=True)
#results = cross_val_score(keras_clf, X_train_prepared, Y_train, cv=kfold)

#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [78]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_train_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

array([[ 963,  564],
       [ 482, 1255]], dtype=int64)

In [79]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


When claiming is a win, correct % is  0.688195991091314
Detected % of wins 0.7115716753022453
F1 score is  0.6996886498726295
Training Accuracy score is  0.6749387254901961


In [80]:
# Train Accuracy
Y_train_pred = keras_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

0.7000612745098039
0.678921568627451


In [81]:
df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Predict.csv")
#df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Predict.csv")

In [82]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T

array([['TBL', 1],
       ['DET', 0],
       ['PIT', 1],
       ['NJD', 1]], dtype=object)